In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
%config Completer.use_jedi = False

In [3]:
df = pd.read_csv('./data/advertising.csv')
df

,TV,radio,newspaper,sales
0,230.1,37.8,69.2,22.1
1,44.5,39.3,45.1,10.4
2,17.2,45.9,69.3,9.3
3,151.5,41.3,58.5,18.5
4,180.8,10.8,58.4,12.9
...,...,...,...,...
195,38.2,3.7,13.8,7.6
196,94.2,4.9,8.1,9.7
197,177.0,9.3,6.4,12.8
198,283.6,42.0,66.2,25.5


### Train-Test Split

0. Clean and adjust the data as needed for X and y
1. Split data into TRAIN and TEST sets for both X and y
2. Fit/train scaler on the TRAINing X data
3. Scale the X TEST data
4. Create the model
5. Fit/train model on the X TRAINing data
6. Evaluate model on X TEST data (by computing predictions and comparing to y TEST)
7. Adjust parameters as needed and repeat steps 5 and 6

In [4]:
X = df.drop('sales', axis=1)
y = df['sales']

In [5]:
from sklearn.model_selection import train_test_split

In [6]:
X_train, X_test, y_train, y_test = train_test_split(
...     X, y, test_size=0.3, random_state=101)

In [7]:
from sklearn.preprocessing import StandardScaler

In [8]:
scaler = StandardScaler()

In [9]:
scaler.fit(X_train)

StandardScaler()

In [10]:
X_train = scaler.transform(X_train)

In [11]:
X_test = scaler.transform(X_test)

In [12]:
from sklearn.linear_model import Ridge

In [13]:
model = Ridge(alpha=100)

In [14]:
model.fit(X_train, y_train)

Ridge(alpha=100)

In [15]:
y_pred = model.predict(X_test)

In [16]:
from sklearn.metrics import mean_squared_error

In [17]:
mean_squared_error(y_test, y_pred)

7.341775789034129

In [18]:
model2 = Ridge(alpha=1)

In [19]:
model2.fit(X_train, y_train)

Ridge(alpha=1)

In [20]:
y_pred2 = model2.predict(X_test)

In [21]:
mean_squared_error(y_test, y_pred2)

2.3190215794287514

### Training-Validation-Test Split

In [22]:
# 70% Training
# 30% Validation and Testing

In [23]:
X = df.drop('sales', axis=1)
y = df['sales']

In [28]:
# FIRST SPLIT: 70% for trainging and 30% for OTHER (to be split again)
X_train, X_other, y_train, y_other = train_test_split(X, y, test_size=0.3, random_state=101)

In [29]:
# SECOND SPLIT
# test_size=0.5 (50% of the 30% "other" from the previous split: therefore, 15% test data from all the data)
X_eval, X_test, y_eval, y_test = train_test_split(X_other, y_other, test_size=0.5, random_state=101)

In [33]:
len(X)

200

In [30]:
len(X_train)

140

In [31]:
len(X_eval)

30

In [32]:
len(X_test)

30

In [35]:
scaler = StandardScaler()
scaler.fit(X_train)

StandardScaler()

In [36]:
X_train = scaler.transform(X_train)
X_eval = scaler.transform(X_eval)
X_test = scaler.transform(X_test)

In [37]:
model_1 = Ridge(alpha=100)
model_1.fit(X_train, y_train)
y_eval_pred = model_1.predict(X_eval)
mean_squared_error(y_eval, y_eval_pred)

7.320101458823872

In [39]:
model_2 = Ridge(alpha=1)
model_2.fit(X_train, y_train)
y_eval_pred = model_2.predict(X_eval)
mean_squared_error(y_eval, y_eval_pred)

2.3837830750569866

In [41]:
# FINAL EVALUATION FOR REPORTING MODEL PERFORMANCE
y_final = model_2.predict(X_test)
mean_squared_error(y_test, y_final)

2.254260083800517

### K-Fold Cross Validation

In [42]:
X = df.drop('sales', axis=1)
y = df['sales']

In [43]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

In [44]:
scaler = StandardScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [61]:
from sklearn.model_selection import cross_val_score

In [66]:
# MODEL 1
model = Ridge(alpha=100)

scores = cross_val_score(model, 
                         X_train, 
                         y_train, 
                         scoring='neg_mean_squared_error', 
                         cv=5)  # CV is number of FOLDS

# positive mean squared error to use for comparison
abs(scores.mean())

8.215396464543607

In [67]:
# MODEL 2
model = Ridge(alpha=1)

scores = cross_val_score(model, 
                         X_train, 
                         y_train, 
                         scoring='neg_mean_squared_error', 
                         cv=5)  # CV is number of FOLDS

# positive mean squared error to use for comparison
abs(scores.mean())

3.344839296530695

In [68]:
# if we are satisfied with our error:
# now fit the model on all the data again

model.fit(X_train, y_train)

Ridge(alpha=1)

In [69]:
y_final = model.predict(X_test)

In [71]:
# ... and then this is the final prediction for reporting the model performance:
mean_squared_error(y_test, y_final)

2.3190215794287514

### Scikit-Learn "cross_validate" Method

In [72]:
# CREATE X AND y
X = df.drop('sales', axis=1)
y = df['sales']

In [73]:
# PERFORM TRAIN-TEST SPLIT
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

In [74]:
# SCALE X DATA
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [75]:
from sklearn.model_selection import cross_validate

In [76]:
model = Ridge(alpha=100)

In [77]:
scores = cross_validate(model, 
                        X_train, 
                        y_train, 
                        scoring=['neg_mean_squared_error', 'neg_mean_absolute_error'],
                        cv=10)

In [79]:
scores = pd.DataFrame(scores)
scores

,fit_time,score_time,test_neg_mean_squared_error,test_neg_mean_absolute_error
0,0.002539,0.001338,-6.060671,-1.810212
1,0.001952,0.002429,-10.627031,-2.541958
2,0.001533,0.001005,-3.993426,-1.469594
3,0.001089,0.000906,-5.009494,-1.862769
4,0.000998,0.000362,-9.141800,-2.520697
5,0.000748,0.000998,-13.086256,-2.459995
6,0.002611,0.001928,-3.839405,-1.451971
7,0.001995,0.000532,-9.058786,-2.377395
8,0.001026,0.001222,-9.055457,-2.443344
9,0.000999,0.001090,-5.778882,-1.899797


In [80]:
scores.mean()

fit_time                        0.001549
score_time                      0.001181
test_neg_mean_squared_error    -7.565121
test_neg_mean_absolute_error   -2.083773
dtype: float64

In [81]:
model = Ridge(alpha=1)

In [82]:
scores = cross_validate(model, 
                        X_train, 
                        y_train, 
                        scoring=['neg_mean_squared_error', 'neg_mean_absolute_error'],
                        cv=10)

scores = pd.DataFrame(scores)
scores.mean()

fit_time                        0.001838
score_time                      0.001274
test_neg_mean_squared_error    -3.323018
test_neg_mean_absolute_error   -1.308467
dtype: float64

In [83]:
model.fit(X_train, y_train)
y_final = model.predict(X_test)
mean_squared_error(y_final, y_test)

2.3190215794287514

### Grid Search

In [85]:
# CREATE X AND y
X = df.drop('sales', axis=1)
y = df['sales']

# PERFORM TRAIN-TEST SPLIT
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

# SCALE X DATA
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [86]:
from sklearn.linear_model import ElasticNet

In [87]:
base_elastic_net_model = ElasticNet()

In [88]:
# optimizing multiple hyperparameters
parameter_grid = {'alpha': [0.1, 1.0, 10.0, 50.0, 100.0],
                  'l1_ratio': [0.1, 0.5, 0.7, 0.95, 0.99, 1.0]}

In [89]:
from sklearn.model_selection import GridSearchCV

In [90]:
grid_model = GridSearchCV(estimator=base_elastic_net_model,
                          param_grid=parameter_grid,
                          scoring='neg_mean_squared_error',
                          cv=5,
                          verbose=2)

In [91]:
grid_model.fit(X_train, y_train)

Fitting 5 folds for each of 30 candidates, totalling 150 fits
[CV] END ............................alpha=0.1, l1_ratio=0.1; total time=   0.0s
[CV] END ............................alpha=0.1, l1_ratio=0.1; total time=   0.0s
[CV] END ............................alpha=0.1, l1_ratio=0.1; total time=   0.0s
[CV] END ............................alpha=0.1, l1_ratio=0.1; total time=   0.0s
[CV] END ............................alpha=0.1, l1_ratio=0.1; total time=   0.0s
[CV] END ............................alpha=0.1, l1_ratio=0.5; total time=   0.0s
[CV] END ............................alpha=0.1, l1_ratio=0.5; total time=   0.0s
[CV] END ............................alpha=0.1, l1_ratio=0.5; total time=   0.0s
[CV] END ............................alpha=0.1, l1_ratio=0.5; total time=   0.0s
[CV] END ............................alpha=0.1, l1_ratio=0.5; total time=   0.0s
[CV] END ............................alpha=0.1, l1_ratio=0.7; total time=   0.0s
[CV] END ............................alpha=0.1,

[CV] END ..........................alpha=100.0, l1_ratio=0.5; total time=   0.0s
[CV] END ..........................alpha=100.0, l1_ratio=0.5; total time=   0.0s
[CV] END ..........................alpha=100.0, l1_ratio=0.5; total time=   0.0s
[CV] END ..........................alpha=100.0, l1_ratio=0.5; total time=   0.0s
[CV] END ..........................alpha=100.0, l1_ratio=0.5; total time=   0.0s
[CV] END ..........................alpha=100.0, l1_ratio=0.7; total time=   0.0s
[CV] END ..........................alpha=100.0, l1_ratio=0.7; total time=   0.0s
[CV] END ..........................alpha=100.0, l1_ratio=0.7; total time=   0.0s
[CV] END ..........................alpha=100.0, l1_ratio=0.7; total time=   0.0s
[CV] END ..........................alpha=100.0, l1_ratio=0.7; total time=   0.0s
[CV] END .........................alpha=100.0, l1_ratio=0.95; total time=   0.0s
[CV] END .........................alpha=100.0, l1_ratio=0.95; total time=   0.0s
[CV] END ...................

GridSearchCV(cv=5, estimator=ElasticNet(),
             param_grid={'alpha': [0.1, 1.0, 10.0, 50.0, 100.0],
                         'l1_ratio': [0.1, 0.5, 0.7, 0.95, 0.99, 1.0]},
             scoring='neg_mean_squared_error', verbose=2)

In [92]:
grid_model.best_estimator_

ElasticNet(alpha=0.1, l1_ratio=1.0)

In [93]:
grid_model.best_params_

{'alpha': 0.1, 'l1_ratio': 1.0}

In [95]:
pd.DataFrame(grid_model.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_alpha,param_l1_ratio,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.014106,0.023715,0.000797,0.000746,0.1,0.1,"{'alpha': 0.1, 'l1_ratio': 0.1}",-3.453021,-1.405190,-5.789125,-2.187302,-4.645576,-3.496043,1.591601,6
1,0.002739,0.000461,0.000809,0.000405,0.1,0.5,"{'alpha': 0.1, 'l1_ratio': 0.5}",-3.325440,-1.427522,-5.595610,-2.163089,-4.451679,-3.392668,1.506827,5
2,0.002322,0.000631,0.001097,0.000148,0.1,0.7,"{'alpha': 0.1, 'l1_ratio': 0.7}",-3.269880,-1.442432,-5.502437,-2.163950,-4.356738,-3.347088,1.462765,4
3,0.003210,0.001246,0.000799,0.000399,0.1,0.95,"{'alpha': 0.1, 'l1_ratio': 0.95}",-3.213052,-1.472417,-5.396258,-2.177452,-4.241080,-3.300052,1.406248,3
4,0.002032,0.000154,0.000890,0.000118,0.1,0.99,"{'alpha': 0.1, 'l1_ratio': 0.99}",-3.208124,-1.478489,-5.380242,-2.181097,-4.222968,-3.294184,1.396953,2
5,0.003919,0.002088,0.000802,0.000748,0.1,1.0,"{'alpha': 0.1, 'l1_ratio': 1.0}",-3.206943,-1.480065,-5.376257,-2.182076,-4.218460,-3.292760,1.394613,1
6,0.005541,0.001415,0.002176,0.000897,1.0,0.1,"{'alpha': 1.0, 'l1_ratio': 0.1}",-9.827475,-5.261525,-11.875347,-7.449195,-8.542329,-8.591174,2.222939,12
7,0.004515,0.000653,0.001677,0.000426,1.0,0.5,"{'alpha': 1.0, 'l1_ratio': 0.5}",-8.707071,-4.214228,-10.879261,-6.204545,-7.173031,-7.435627,2.255532,11
8,0.004642,0.000499,0.001517,0.000605,1.0,0.7,"{'alpha': 1.0, 'l1_ratio': 0.7}",-7.920870,-3.549562,-10.024877,-5.379553,-6.324836,-6.639940,2.206213,10
9,0.005076,0.000706,0.001693,0.000484,1.0,0.95,"{'alpha': 1.0, 'l1_ratio': 0.95}",-6.729435,-2.591285,-8.709842,-4.156317,-5.329916,-5.503359,2.102835,9


In [96]:
y_pred = grid_model.predict(X_test)

In [97]:
mean_squared_error(y_test, y_pred)

2.3873426420874737